In [245]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [246]:
symbol = 'spy'
data = vbt.AlpacaData.download(symbol,
                               start='2014-3-05',
                               end='2019-3-16',
                               # limit=1000,
                               timeframe='1d').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'close': array([185.59, 183.69, 181.12, ..., 235.94, 237.45, 237.32]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([185.71, 185.87, 184.24, ..., 236.17, 237.7 , 237.65]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([184.21, 183.43, 180.37, ..., 235.56, 235.93, 237.12]),
              'name': 'Candlestick',
              'open': array([184.5 , 185.53, 183.96, ..., 235.81, 236.76, 237.51]),
              'type': 'candlestick',
              'uid': '6b70c3b3-bf42-4a8e-a055-13986d551cc6',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2017, 11, 20, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 11, 21, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 11, 22, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'yaxis': 'y'},
             {'marker': {'color': array(['#1b9e76', '#d95f02', '#d95f02', ..., '#1b9e76', '#1b9e76', '#d95f02'],
                                        dtype=object),
                         'line': {'width': 0}},
              'name': 'Volume',
              'opacity': 0.5,
              'type': 'bar',
              'uid': '6fe87308-f356-461e-84e2-f6a604f193ea',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2017, 11, 20, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 11, 21, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 11, 22, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x2',
              'y': array([9.78584180e+07, 1.08069059e+08, 1.66224154e+08, ..., 4.80904010e+07,
                          6.95731790e+07, 4.50333920e+07]),
              'yaxis': 'y2'}],
    'layout': {'bargap': 0,
               'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'showlegend': True,
               'template': '...',
               'width': 700,
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'matches': 'x2',
                         'rangeslider': {'visible': False},
                         'showgrid': True,
                         'showticklabels': False},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0], 'showgrid': True},
               'yaxis': {'anchor': 'x', 'domain': [0.3, 1.0], 'showgrid': True},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.3], 'showgrid': True}}
})

In [247]:
def pullback_place_entry(start, close, rsi_window = 10, sma_window = 200, entry = 30):
    rsi = vbt.RSI.run(close, rsi_window).rsi.to_numpy()
    sma = vbt.MA.run(close, sma_window).ma.to_numpy()


    entry = np.where((rsi < entry) & (sma < start), 1, 0)

    return entry, rsi, sma

PULLBACK_PLACE_ENTRY = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['start', 'close'],
    output_names = ['value', 'rsi', 'sma'],
    param_names=['rsi_window', 'sma_window', 'entry']
).from_apply_func(pullback_place_entry, rsi_window = 10, sma_window = 200, entry = 30)

enter = PULLBACK_PLACE_ENTRY.run(data['Open'], data['Close'], rsi_window = 10, sma_window = 200, entry = 30)

data['entry'] = enter.value
data['rsi'] = enter.rsi
data['sma'] = enter.sma

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

,Open,High,Low,Close,Volume,entry,rsi,sma
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,184.50,185.71,184.21,185.59,97858418.0,0,NaN,NaN
2015-12-02 05:00:00+00:00,185.53,185.87,183.43,183.69,108069059.0,0,NaN,NaN
2015-12-03 05:00:00+00:00,183.96,184.24,180.37,181.12,166224154.0,0,NaN,NaN
2015-12-04 05:00:00+00:00,181.54,184.96,181.40,184.66,192878747.0,0,NaN,NaN
2015-12-07 05:00:00+00:00,184.31,184.75,182.52,183.54,102027111.0,0,NaN,NaN
2015-12-08 05:00:00+00:00,181.90,183.48,181.27,182.30,103372367.0,0,NaN,NaN
2015-12-09 05:00:00+00:00,181.63,183.83,179.86,180.89,162401537.0,0,NaN,NaN
2015-12-10 05:00:00+00:00,180.96,182.73,180.71,181.35,116128858.0,0,NaN,NaN
2015-12-11 05:00:00+00:00,179.13,179.83,177.51,177.84,211173305.0,0,NaN,NaN


In [248]:
def pullback_place_exit(enter, close, rsi_window = 10, out = 40, leave = 10):
    rsi = vbt.RSI.run(close, rsi_window).rsi.to_numpy()


    leave_array = np.roll(enter.astype(float), leave)
    leave_array[:leave] = np.inf
    #print(leave_array)

    stop_loss =  np.where((leave_array == 1), -1, 0)
    take_profit = np.where(rsi>out, -1, 0)
    sell = np.where((leave_array == 1) | (rsi>out) , -1, 0)

    return take_profit, stop_loss, sell

PULLBACK_PLACE_EXIT = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['enter', 'close'],
    output_names = ['take_profit', 'stop_loss', 'sell'],
    param_names=['rsi_window', 'out', 'leave']
).from_apply_func(pullback_place_exit, rsi_window=10, out = 40, leave=10)

exit = PULLBACK_PLACE_EXIT.run(enter.value, data['Close'], rsi_window = 10, out= 40, leave=10)

data['stop_loss'] = exit.stop_loss
data['take_profit'] = exit.take_profit
data['sell']=exit.sell


with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)


exits = exit.sell == -1
entries = enter.value == 1

entries = np.roll(entries, 2)
exits = np.roll(exits, 2)

pf = vbt.Portfolio.from_signals(data['Open'], entries, exits, init_cash=10000, freq='1d')

orders = pf.orders





,Open,High,Low,Close,Volume,entry,rsi,sma,stop_loss,take_profit,sell
timestamp,,,,,,,,,,,
2015-12-01 05:00:00+00:00,184.50,185.71,184.21,185.59,97858418.0,0,NaN,NaN,0,0,0
2015-12-02 05:00:00+00:00,185.53,185.87,183.43,183.69,108069059.0,0,NaN,NaN,0,0,0
2015-12-03 05:00:00+00:00,183.96,184.24,180.37,181.12,166224154.0,0,NaN,NaN,0,0,0
2015-12-04 05:00:00+00:00,181.54,184.96,181.40,184.66,192878747.0,0,NaN,NaN,0,0,0
2015-12-07 05:00:00+00:00,184.31,184.75,182.52,183.54,102027111.0,0,NaN,NaN,0,0,0
2015-12-08 05:00:00+00:00,181.90,183.48,181.27,182.30,103372367.0,0,NaN,NaN,0,0,0
2015-12-09 05:00:00+00:00,181.63,183.83,179.86,180.89,162401537.0,0,NaN,NaN,0,0,0
2015-12-10 05:00:00+00:00,180.96,182.73,180.71,181.35,116128858.0,0,NaN,NaN,0,0,0
2015-12-11 05:00:00+00:00,179.13,179.83,177.51,177.84,211173305.0,0,NaN,NaN,0,0,0


In [249]:
pf.stats()


Start                         2015-12-01 05:00:00+00:00
End                           2017-11-22 05:00:00+00:00
Period                                500 days 00:00:00
Start Value                                     10000.0
End Value                                  10458.894313
Total Return [%]                               4.588943
Benchmark Return [%]                          28.731707
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.686805
Max Drawdown Duration                  13 days 00:00:00
Total Trades                                          5
Total Closed Trades                                   5
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       80.0
Best Trade [%]                                 3.489176
Worst Trade [%]                               -0

In [250]:
orders.plot()



AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'db12f46e-42dc-42b7-876f-ff2fa0990dc9',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2017, 11, 20, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 11, 21, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 11, 22, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([184.5 , 185.53, 183.96, ..., 235.81, 236.76, 237.51])},
             {'customdata': array([[ 0.        , 52.12405525,  0.        ],
                                   [ 2.        , 53.09162587,  0.        ],
                                   [ 4.        , 48.86026699,  0.        ],
                                   [ 6.        , 46.43301219,  0.        ],
                                   [ 8.        , 46.96194295,  0.        ]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy',
              'type': 'scatter',
              'uid': '9f7bf821-1ab6-45a4-9e5e-8ade5f1eeb12',
              'x': array([datetime.datetime(2016, 10, 18, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 11, 3, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 4, 18, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 14, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 23, 4, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([191.85, 188.01, 211.42, 223.24, 222.1 ])},
             {'customdata': array([[ 1.        , 52.12405525,  0.        ],
                                   [ 3.        , 53.09162587,  0.        ],
                                   [ 5.        , 48.86026699,  0.        ],
                                   [ 7.        , 46.43301219,  0.        ],
                                   [ 9.        , 46.96194295,  0.        ]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'marker': {'color': '#EA4335',
                         'line': {'color': 'rgb(181,31,18)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-down'},
              'mode': 'markers',
              'name': 'Sell',
              'type': 'scatter',
              'uid': 'cffb02eb-dec2-4ad7-97b3-99958510b95a',
              'x': array([datetime.datetime(2016, 10, 20, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2016, 11, 10, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 4, 19, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 16, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 24, 4, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([191.5 , 194.57, 212.15, 224.63, 222.71])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                  